## **Dependencies**

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import time

from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random
import torchsummary
from torchsummary import summary

print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cuda')
print(device)

1.7.1
cpu


## **Hyper parameters**

In [2]:
# batch_size = 60
batch_size = 64
random_seed = 10
initial_lr = 0.1
# num_epoch = 250
num_epoch = 5
# num_images = 12620
# num_images = 12620
# split = round(num_images/5)

## **Dataset split & Class speration**

In [3]:
transform_train = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_validation = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_test = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.ImageFolder(
root='/Users/maisie_tang/Documents/career/Picogrid/wildfire-smoke-detection-camera/train/', transform=transform_train)
validset = torchvision.datasets.ImageFolder(
root='/Users/maisie_tang/Documents/career/Picogrid/wildfire-smoke-detection-camera/train/', transform=transform_validation)
testset = torchvision.datasets.ImageFolder(
root='/Users/maisie_tang/Documents/career/Picogrid/wildfire-smoke-detection-camera/test/', transform=transform_test)

num_train = len(trainset)
indices = list(range(num_train))
np.random.seed(random_seed)
np.random.shuffle(indices)

split = 450
#train_idx_1 = indices[:1346]
#train_idx_2 = indices[2693:]
#train_idx = train_idx_1 + train_idx_2

# train_idx = indices[split+1:]
train_idx = indices[split+1:4301]
valid_idx = indices[:split]

num_test = len(testset)
indices_test = list(range(num_test))
test_idx = indices[:]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, sampler=train_sampler, num_workers=4
)

valid_loader = torch.utils.data.DataLoader(
    validset, batch_size=batch_size, sampler=valid_sampler, num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    testset, batch_size=28, shuffle=False, num_workers=4
)

classes = ('wildfire','nonfire')

## **DenseNet architecture**

In [4]:
class bn_relu_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size, stride, padding, bias=False):
        super(bn_relu_conv, self).__init__()
        self.batch_norm = nn.BatchNorm2d(nin)
        self.relu = nn.ReLU(True)
        self.conv = nn.Conv2d(nin, nout, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

    def forward(self, x):
        out = self.batch_norm(x)
        out = self.relu(out)
        out = self.conv(out)

        return out

class bottleneck_layer(nn.Sequential):
    def __init__(self, nin, growth_rate, drop_rate=0.2):    
        super(bottleneck_layer, self).__init__()
      
        self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=growth_rate*4, kernel_size=1, stride=1, padding=0, bias=False))
        self.add_module('conv_3x3', bn_relu_conv(nin=growth_rate*4, nout=growth_rate, kernel_size=3, stride=1, padding=1, bias=False))
      
        self.drop_rate = drop_rate
      
    def forward(self, x):
        bottleneck_output = super(bottleneck_layer, self).forward(x)
        if self.drop_rate > 0:
            bottleneck_output = F.dropout(bottleneck_output, p=self.drop_rate, training=self.training)
          
        bottleneck_output = torch.cat((x, bottleneck_output), 1)
      
        return bottleneck_output

class Transition_layer(nn.Sequential):
    def __init__(self, nin, theta=0.5):    
        super(Transition_layer, self).__init__()
      
        self.add_module('conv_1x1', bn_relu_conv(nin=nin, nout=int(nin*theta), kernel_size=1, stride=1, padding=0, bias=False))
        self.add_module('avg_pool_2x2', nn.AvgPool2d(kernel_size=2, stride=2, padding=0))

class DenseBlock(nn.Sequential):
    def __init__(self, nin, num_bottleneck_layers, growth_rate, drop_rate=0.2):
        super(DenseBlock, self).__init__()
                        
        for i in range(num_bottleneck_layers):
            nin_bottleneck_layer = nin + growth_rate * i
            self.add_module('bottleneck_layer_%d' % i, bottleneck_layer(nin=nin_bottleneck_layer, growth_rate=growth_rate, drop_rate=drop_rate))

class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, num_layers=10, theta=0.5, drop_rate=0.2, num_classes=10):
        super(DenseNet, self).__init__()

        assert (num_layers - 4) % 6 == 0

        num_bottleneck_layers = (num_layers - 4) // 6

        self.dense_init = nn.Conv2d(3, growth_rate*2, kernel_size=7, stride=2, padding=3, bias=True)
        self.dense_init_2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.dense_block_1 = DenseBlock(nin=growth_rate*2, num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        nin_transition_layer_1 = (growth_rate*2) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_1 = Transition_layer(nin=nin_transition_layer_1, theta=theta)

        self.dense_block_2 = DenseBlock(nin=int(nin_transition_layer_1*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        nin_transition_layer_2 = int(nin_transition_layer_1*theta) + (growth_rate * num_bottleneck_layers) 
        self.transition_layer_2 = Transition_layer(nin=nin_transition_layer_2, theta=theta)

        self.dense_block_3 = DenseBlock(nin=int(nin_transition_layer_2*theta), num_bottleneck_layers=num_bottleneck_layers, growth_rate=growth_rate, drop_rate=drop_rate)

        nin_fc_layer = int(nin_transition_layer_2*theta) + (growth_rate * num_bottleneck_layers) 

        self.fc_layer = nn.Linear(nin_fc_layer, num_classes)

    def forward(self, x):
        dense_init_output = self.dense_init(x)
        dense_init_output_2 = self.dense_init_2(dense_init_output)

        dense_block_1_output = self.dense_block_1(dense_init_output_2)
        transition_layer_1_output = self.transition_layer_1(dense_block_1_output)

        dense_block_2_output = self.dense_block_2(transition_layer_1_output)
        transition_layer_2_output = self.transition_layer_2(dense_block_2_output)

        dense_block_3_output = self.dense_block_3(transition_layer_2_output)

        global_avg_pool_output = F.adaptive_avg_pool2d(dense_block_3_output, (1, 1))                
        global_avg_pool_output_flat = global_avg_pool_output.view(global_avg_pool_output.size(0), -1)

        output = self.fc_layer(global_avg_pool_output_flat)

        return output
    
# def Optimized_DenseNet():
#     return DenseNet(growth_rate=24, num_layers=100, theta=0.5, drop_rate=0.2, num_classes=2)

def Optimized_DenseNet():
    return DenseNet(growth_rate=24, num_layers=10, theta=0.5, drop_rate=0.2, num_classes=2)

net = Optimized_DenseNet()
densenet = net.to(device)
densenet

DenseNet(
  (dense_init): Conv2d(3, 48, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (dense_init_2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dense_block_1): DenseBlock(
    (bottleneck_layer_0): bottleneck_layer(
      (conv_1x1): bn_relu_conv(
        (batch_norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (conv_3x3): bn_relu_conv(
        (batch_norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv): Conv2d(96, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
  )
  (transition_layer_1): Transition_layer(
    (conv_1x1): bn_relu_conv(
      (batch_norm): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)


## **Model summary**

In [5]:
torchsummary.summary(net, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 112, 112]           7,104
         MaxPool2d-2           [-1, 48, 56, 56]               0
       BatchNorm2d-3           [-1, 48, 56, 56]              96
              ReLU-4           [-1, 48, 56, 56]               0
            Conv2d-5           [-1, 96, 56, 56]           4,608
      bn_relu_conv-6           [-1, 96, 56, 56]               0
       BatchNorm2d-7           [-1, 96, 56, 56]             192
              ReLU-8           [-1, 96, 56, 56]               0
            Conv2d-9           [-1, 24, 56, 56]          20,736
     bn_relu_conv-10           [-1, 24, 56, 56]               0
      BatchNorm2d-11           [-1, 72, 56, 56]             144
             ReLU-12           [-1, 72, 56, 56]               0
           Conv2d-13           [-1, 36, 56, 56]           2,592
     bn_relu_conv-14           [-1, 36,

## **Training & Save the trained model**

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=initial_lr, momentum=0.9)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[int(num_epoch * 0.5), int(num_epoch * 0.75)], gamma=0.1, last_epoch=-1)

for epoch in range(num_epoch):  
    lr_scheduler.step()
    
    running_loss = 0.0
    epoch_loss, epoch_time = 0, 0
    for i, data in enumerate(train_loader, 0):
        s_time = time.time()


        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        show_period = 180
        if i % show_period == show_period-1:    # print every "show_period" mini-batches
           
            running_loss = 0.0
        
        ### DEBUG
        e_time = time.time()
        batch_time = e_time - s_time
        epoch_loss += loss.item()
        print(f'epoch({epoch:03d}) batch ({i:03d}) loss: {loss.item():0.5f} ({batch_time:4.1f})')
        
        
    # validation part
    correct = 0
    total = 0
    for i, data in enumerate(valid_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    


print('Finished Training')


path = "/Users/maisie_tang/Documents/career/Picogrid/savemodel/model2.pth"
torch.save(net.state_dict(),path)


/Users/maisie_tang/opt/anaconda3/envs/pandas_playground/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch(000) batch (000) loss: 0.68582 ( 2.5)
epoch(000) batch (001) loss: 0.70410 ( 1.9)
epoch(000) batch (002) loss: 0.64030 ( 2.0)
epoch(000) batch (003) loss: 0.72755 ( 2.0)
epoch(000) batch (004) loss: 0.65305 ( 2.0)
epoch(000) batch (005) loss: 0.65920 ( 2.0)
epoch(000) batch (006) loss: 0.56103 ( 1.9)
epoch(000) batch (007) loss: 0.56078 ( 1.9)
epoch(000) batch (008) loss: 0.61485 ( 2.0)
epoch(000) batch (009) loss: 0.55693 ( 2.0)
epoch(000) batch (010) loss: 0.58210 ( 2.0)
epoch(000) batch (011) loss: 0.57934 ( 2.0)
epoch(000) batch (012) loss: 0.73924 ( 2.2)
epoch(000) batch (013) loss: 0.55141 ( 2.0)
epoch(000) batch (014) loss: 0.56469 ( 2.0)
epoch(000) batch (015) loss: 0.52456 ( 2.0)
epoch(000) batch (016) loss: 0.39980 ( 2.1)
epoch(000) batch (017) loss: 0.53175 ( 3.1)
epoch(000) batch (018) loss: 0.49647 ( 3.7)
epoch(000) batch (019) loss: 0.49401 ( 3.7)
epoch(000) batch (020) loss: 0.60499 ( 2.6)
epoch(000) batch (021) loss: 0.47321 ( 2.9)
epoch(000) batch (022) loss: 0.5

epoch(003) batch (004) loss: 0.33475 ( 2.0)
epoch(003) batch (005) loss: 0.40843 ( 2.0)
epoch(003) batch (006) loss: 0.34346 ( 2.0)
epoch(003) batch (007) loss: 0.35319 ( 2.0)
epoch(003) batch (008) loss: 0.31031 ( 2.0)
epoch(003) batch (009) loss: 0.24246 ( 2.0)
epoch(003) batch (010) loss: 0.36022 ( 2.0)
epoch(003) batch (011) loss: 0.30893 ( 2.0)
epoch(003) batch (012) loss: 0.29491 ( 2.0)
epoch(003) batch (013) loss: 0.30347 ( 2.1)
epoch(003) batch (014) loss: 0.46521 ( 2.1)
epoch(003) batch (015) loss: 0.44617 ( 2.1)
epoch(003) batch (016) loss: 0.31229 ( 2.2)
epoch(003) batch (017) loss: 0.37680 ( 2.1)
epoch(003) batch (018) loss: 0.34167 ( 2.1)
epoch(003) batch (019) loss: 0.19208 ( 2.0)
epoch(003) batch (020) loss: 0.32790 ( 2.3)
epoch(003) batch (021) loss: 0.30819 ( 2.3)
epoch(003) batch (022) loss: 0.28903 ( 2.1)
epoch(003) batch (023) loss: 0.45504 ( 2.1)
epoch(003) batch (024) loss: 0.29334 ( 2.1)
epoch(003) batch (025) loss: 0.32501 ( 2.2)
epoch(003) batch (026) loss: 0.2

## **Load model & test**

In [7]:
path = "/Users/maisie_tang/Documents/career/Picogrid/savemodel/model2.pth"
net.load_state_dict(torch.load(path))
net.eval()

class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        
        _, predicted = torch.max(outputs,1)
        c = (predicted == labels).squeeze()
        print(torch.nn.functional.softmax(outputs, dim=1))
        print(predicted)     
        for i in range(labels.shape[0]):
            
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


print('Accuracy of the network on the 1031 test images: %5s %%' % (100 * correct / total))            
            
for i in range(2):
    print('Accuracy of %5s : %5s %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
    print(class_correct[i])
    print(class_total[i])

tensor([[0.8430, 0.1570],
        [0.8200, 0.1800],
        [0.8634, 0.1366],
        [0.8034, 0.1966],
        [0.8590, 0.1410],
        [0.7853, 0.2147],
        [0.8184, 0.1816],
        [0.7154, 0.2846],
        [0.9753, 0.0247],
        [0.9935, 0.0065],
        [0.9709, 0.0291],
        [0.7141, 0.2859],
        [0.9707, 0.0293],
        [0.9880, 0.0120],
        [0.9750, 0.0250],
        [0.6182, 0.3818],
        [0.9431, 0.0569],
        [0.7669, 0.2331],
        [0.9863, 0.0137],
        [0.9766, 0.0234],
        [0.9516, 0.0484],
        [0.7184, 0.2816],
        [0.9636, 0.0364],
        [0.9769, 0.0231],
        [0.9647, 0.0353],
        [0.7620, 0.2380],
        [0.9458, 0.0542],
        [0.8992, 0.1008]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
tensor([[0.8403, 0.1597],
        [0.8765, 0.1235],
        [0.8900, 0.1100],
        [0.8861, 0.1139],
        [0.8123, 0.1877],
        [0.8041, 0.1959],
        [0.869

tensor([[0.0693, 0.9307],
        [0.1534, 0.8466],
        [0.8625, 0.1375],
        [0.0021, 0.9979],
        [0.0020, 0.9980],
        [0.8381, 0.1619],
        [0.0059, 0.9941],
        [0.0101, 0.9899],
        [0.7917, 0.2083],
        [0.0022, 0.9978],
        [0.0024, 0.9976],
        [0.9251, 0.0749],
        [0.0250, 0.9750],
        [0.0470, 0.9530],
        [0.7799, 0.2201],
        [0.0021, 0.9979],
        [0.0021, 0.9979],
        [0.7731, 0.2269],
        [0.0021, 0.9979],
        [0.0020, 0.9980],
        [0.8927, 0.1073],
        [0.0367, 0.9633],
        [0.0673, 0.9327],
        [0.9421, 0.0579],
        [0.0100, 0.9900],
        [0.0067, 0.9933],
        [0.8706, 0.1294],
        [0.1966, 0.8034]])
tensor([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 1])
tensor([[0.1735, 0.8265],
        [0.8947, 0.1053],
        [0.0062, 0.9938],
        [0.0048, 0.9952],
        [0.7548, 0.2452],
        [0.1004, 0.8996],
        [0.224

## **Class activation map**

In [8]:
class CAM(nn.Module):
    def __init__(self, model_to_convert, get_fc_layer=lambda m: m.fc_layer,score_fn=F.softmax, resize=True):
        super().__init__()
        self.backbone = nn.Sequential(*list(model_to_convert.children())[:-1])
        self.fc = get_fc_layer(model_to_convert)
        self.conv  =  nn.Conv2d(self.fc.in_features, self.fc.out_features, kernel_size=1)
        self.conv.weight = nn.Parameter(self.fc.weight.data.unsqueeze(-1).unsqueeze(-1))
        self.conv.bias = self.fc.bias
        self.score_fn = score_fn
        self.resize = resize
        self.eval()
        
    def forward(self, x, out_size=None):
        batch_size, c, *size = x.size()
        feat = self.backbone(x)
        cmap = self.score_fn(self.conv(feat))
        if self.resize:
            if out_size is None:
                out_size = size
            cmap = F.upsample(cmap, size=out_size, mode='bicubic')
        pooled = F.adaptive_avg_pool2d(feat,output_size=1)
        flatten = pooled.view(batch_size, -1)
        cls_score = self.score_fn(self.fc(flatten))
        weighted_cmap =  (cmap*cls_score.unsqueeze(-1).unsqueeze(-1)).sum(dim=1)
        return cmap, cls_score, weighted_cmap
    
path = "/Users/maisie_tang/Documents/career/Picogrid/savemodel/model1.pth"
net.load_state_dict(torch.load(path))
cam = CAM(net)
#assert not cam.training

if torch.cuda.is_available():
    print("use gpu")
    cam = cam.cuda()
    def to_var(x, requires_grad=False, volatile=False):
        return Variable(x.cuda(), requires_grad=requires_grad, volatile=volatile)
else:
    def to_var(x, requires_grad=False, volatile=False):
        return Variable(x, requires_grad=requirs_grad, volatile=volatile)


target_size = (224,224)

normalize = transforms.Normalize([0.5, 0.5, 0.5],
                                 [0.5, 0.5, 0.5])
transform = transforms.Compose([transforms.Scale(target_size),transforms.CenterCrop(target_size),
                                transforms.ToTensor()])
from torch.autograd import Variable

img_path = "/Users/maisie_tang/Documents/career/Picogrid/wildfire-smoke-detection-camera/test/wildfire/ck0k9etuqjxhh0848k6i1mw2f.jpeg"
img = Image.open(img_path)
img_v = to_var(transform(img).unsqueeze(0),volatile=True)

cmap, score, weighted_cmap = cam(img_v)
print(cmap.size())
print(score.size())
print(weighted_cmap.size())

import matplotlib.pyplot as plt
background = np.array(img.resize(target_size))
color_map = weighted_cmap.data.cpu().numpy()[0]
color_map = cmap.data.cpu().numpy()[0,1]
#print(color_map)
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.imshow(background)
plt.imshow(color_map,cmap ='jet',alpha=0.5)
plt.show()

/Users/maisie_tang/opt/anaconda3/envs/pandas_playground/lib/python3.8/site-packages/torchvision/transforms/transforms.py:187: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


NameError: name 'requirs_grad' is not defined